# Arrays

## 1. Setup

In [1]:
# Standard library imports
from typing import Union

# Third party imports
import numpy as np
import torch
from torch import nn

## 2. Scenarios

### 2.1 Images

You have a six pixel image represnted in the format:
```
Alpha:
{
    R G       R G       R G
    B A       B A       B A

    R G       R G       R G
    B A       B A       B A
}

Beta:
{
    11 200    19 16     40 23
    18 0.2    99 0.25   12 0.4

    32 184    26 177    35 99
    12 0.8    6 0.75    7 0.9
}

```

#### 2.1.1 Data

In [2]:
# Red = 0
# Blue = 1
# Green = 2
# Alpha = 3

alpha_simple = np.array([
    [[[0, 1, 2, 3]] * 3] * 2,
])[0]

In [3]:
alpha_simple

array([[[0, 1, 2, 3],
        [0, 1, 2, 3],
        [0, 1, 2, 3]],

       [[0, 1, 2, 3],
        [0, 1, 2, 3],
        [0, 1, 2, 3]]])

In [4]:
beta_simple = np.array([
    [[11, 200, 18, 0.2], [19, 16, 99, 0.25], [40, 23, 12, 0.4]],
    [[32, 184, 12, 0.8], [26, 177, 6, 0.75], [35, 99, 7, 0.9]], 
])

In [5]:
print(f"Array has: {beta_simple.shape[0]} rows")
print(f"Array has: {beta_simple.shape[1]} columns")
print(f"Each row/column combination has: {beta_simple.shape[2]} values")

Array has: 2 rows
Array has: 3 columns
Each row/column combination has: 4 values


In [6]:
alpha_flat = np.array(
    [[[0, 1]] * 3 , [[2, 3]] * 3] * 2
).reshape(-1, 6)

In [7]:
alpha_flat

array([[0, 1, 0, 1, 0, 1],
       [2, 3, 2, 3, 2, 3],
       [0, 1, 0, 1, 0, 1],
       [2, 3, 2, 3, 2, 3]])

In [8]:
beta_flat = np.array([
    [11, 200, 19, 16, 40, 23],
    [18, 0.2, 99, 0.25, 12, 0.4],
    [32, 184, 26, 177, 35, 99], 
    [ 12, 0.8, 6, 0.75, 7, 0.9], 
])

#### 2.1.2 Functions

In [9]:
def via_mask(alpha: np.ndarray, beta: np.ndarray, chosen_alpha: Union[float, int]) -> np.ndarray:
    """Via masking.

    Args:
        alpha (np.ndarray): Array mapping values of beta to a colour.
        beta (np.ndarray): Array of values.
        colour (str): Chosen value of alpha.

    Returns:
        np.ndarray: Corresponding values.
    """
    # Create mask
    mask = alpha == chosen_alpha

    output = beta[mask]

    return output

In [10]:
def via_max_pooling(alpha: np.ndarray, beta: np.ndarray, chosen_alpha: Union[float, int]) -> np.ndarray:
    """Via max pooling.

    Args:
        alpha (np.ndarray): Array mapping values of beta to a colour.
        beta (np.ndarray): Array of values.
        colour (str): Chosen value of alpha.

    Returns:
        np.ndarray: Corresponding values.

    Extended Summary:
        This function will not work in all cases but has been written to
        demonstrate how max-pooling of an array works.
    """
    # Create a mask
    mask = np.ma.masked_equal(alpha, chosen_alpha).mask

    # Copy beta (to prevent overwriting the original data)
    beta_copy = beta.copy()

    # Set values other than the chosen alpha to -1
    beta_copy[~mask] = -1

    # This step will depend on the shape of the input data!
    if len(alpha.shape) == 2:
        beta_transformed = np.expand_dims(beta_copy, axis=0)
    else:
        raise NotImplementedError("Code assumes alpha/beta is 2D")

    # Create torch from numpy array
    beta_tensor = torch.from_numpy(beta_transformed.astype(float))

    # Run max pooling with a window of size (2, 2) with a step/stride such that
    # there is no overlapping
    m = nn.MaxPool2d((2, 2), stride=(2, 2))
    output = m(beta_tensor).detach().numpy()
    return output

#### 2.1.3 Testing 'simple' data

In [11]:
via_mask(alpha=alpha_simple, beta=beta_simple, chosen_alpha=0).reshape((beta_simple.shape[0], beta_simple.shape[1]))

array([[11., 19., 40.],
       [32., 26., 35.]])

In [12]:
via_mask(alpha=alpha_simple, beta=beta_simple, chosen_alpha=1).reshape((beta_simple.shape[0], beta_simple.shape[1]))

array([[200.,  16.,  23.],
       [184., 177.,  99.]])

In [13]:
via_mask(alpha=alpha_simple, beta=beta_simple, chosen_alpha=2).reshape((beta_simple.shape[0], beta_simple.shape[1]))

array([[18., 99., 12.],
       [12.,  6.,  7.]])

In [14]:
via_mask(alpha=alpha_simple, beta=beta_simple, chosen_alpha=3).reshape((beta_simple.shape[0], beta_simple.shape[1]))

array([[0.2 , 0.25, 0.4 ],
       [0.8 , 0.75, 0.9 ]])

#### 2.1.4 Testing 'flat' data

In [15]:
via_mask(alpha=alpha_flat, beta=beta_flat, chosen_alpha=1).reshape(-1, 3)

array([[200.,  16.,  23.],
       [184., 177.,  99.]])

In [16]:
via_max_pooling(alpha=alpha_flat, beta=beta_flat, chosen_alpha=1)[0]

array([[200.,  16.,  23.],
       [184., 177.,  99.]])